# We extract all denner zurich branches location


In [164]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [12]:
with open("../data/denner_branches/Filialen.html", 'r') as f:
    soup = BeautifulSoup(f.read(), 'html')

In [15]:
type(soup)

bs4.BeautifulSoup

In [37]:
stores = soup.findAll('div', {'role' : 'tabpanel'})
# {'id' : 'collapse-'}

In [38]:
len(stores)

160

### Address:

In [238]:

stores[0].findAll('span')[0].text

' Wengistrasse 7'

In [239]:
stores[0].findAll('span')[1].text

'8004 Zürich'

In [58]:
stores[0].findAll('span')

[<span> Wengistrasse 7</span>,
 <span>8004 Zürich</span>,
 <span itemprop="name">Denner Express Zürich</span>,
 <span itemprop="streetAddress">Wengistrasse 7</span>,
 <span itemprop="addressLocality">Zürich</span>,
 <span itemprop="postalCode">8004</span>,
 <span class="validFromDate" date="2023-12-06" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-07" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-08" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-09" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-10" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-11" itemprop="validFrom validThrough"></span>,
 <span class="validFromDate" date="2023-12-12" itemprop="validFrom validThrough"></span>]

In [56]:
stores[50].findAll('span', {'itemprop' : 'postalCode'})[0].text

'8134'

Type:

Type of store:

In [158]:
stores[0].findAll('dd')[1].text.strip()

'Denner Express'

Link to google maps

In [100]:
stores[159].find('a', {'class' : 'google_track_link'}).get('href')

'https://maps.google.com/maps?saddr&daddr=W%C3%A4chlenstrasse%201,%208832,Wollerau,%20Switzerland'

Telephone

In [128]:
 i =stores[0].findAll('dd')[0].text.find("Tel.")

In [136]:
stores[0].findAll('dd')[0].text[i+5:i+18]

'058 999 06 27'

Opening time

In [219]:
day = stores[58].findAll('tr', {'itemprop' : 'openingHoursSpecification'})
len(day)

13

In [189]:
day[9]

<tr itemprop="openingHoursSpecification" itemscope="" itemtype="http://schema.org/OpeningHoursSpecification">
<td class="daylabel">
											
												Di, 26.12.2023
												<time content="26.12.2023" itemprop="validFrom"></time>
<time content="26.12.2023" itemprop="validThrough"></time>
</td>
<td>
<strong>
<time content="00:00" itemprop="opens"></time>geschlossen<time content="00:00" itemprop="closes"></time>
												
											(Stephanstag)
											</strong>
</td>
</tr>

In [199]:
day[2].find("td", {"itemprop": "dayOfWeek"}).text.split()[0]

'Freitag'

In [174]:
day[4].find("time", {"itemprop": "opens"})

In [149]:
day.find("time", {"itemprop": "closes"}).text

'20:00'

Angebot

In [ ]:
/html/body/div[8]/div[3]/div/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/dl/dd/div[2]

In [210]:
stores[4].findAll('dd')[5].findAll("div")[1].text

'Humidor, Bargeldbezug mit Post- / M-Card'

In [214]:
stores[4].findAll('dd')[5].findAll("div")[1].text

'Humidor, Bargeldbezug mit Post- / M-Card'

In [225]:
day[4].find("td", {"itemprop": "dayOfWeek"}).text.split()[0]

'Sonntag'

In [231]:
def get_filialen(stores):
    cols = ["street_number", "post_municipality", "telephone", "type_store", "Angebot", "Montag", "Dienstag", "Mittwoch", "Donnerstag", "Freitag", "Samstag", "Sonntag" ,"gmaps_link"]
    df = pd.DataFrame(columns = cols)
    daysOfWeek = ["Montag", "Dienstag", "Mittwoch", "Donnerstag", "Freitag", "Samstag", "Sonntag" ]
    for i in range(len(stores)):
        try:
            street_number = stores[i].findAll('span')[0].text
        except AttributeError:
            street_number = np.nan

        try:
            post_municipality = stores[i].findAll('span')[1].text
        except AttributeError:
            post_municipality = np.nan

        try:
            type_store = stores[i].findAll('dd')[1].text.strip()
        except AttributeError:
            type_store = np.nan
        try:
            j =stores[i].findAll('dd')[0].text.find("Tel.")
            telephone = stores[i].findAll('dd')[0].text[j+5:j+18]
        except AttributeError:
            telephone = np.nan
        try:
            # print("Pedro prints", i)
            try:
                Angebot = stores[i].findAll('dd')[4].findAll("div")[1].text
            except IndexError:
                try:
                    Angebot = stores[i].findAll('dd')[5].findAll("div")[1].text
                except:
                    Angebot = np.nan
        except AttributeError:
            Angebot = np.nan
            
        try:
            gmaps_link = stores[i].find('a', {'class' : 'google_track_link'}).get('href')
        except AttributeError:
            gmaps_link = np.nan
        
        try:
            days = stores[i].findAll('tr', {'itemprop' : 'openingHoursSpecification'})
        except AttributeError:
            days = 0

        if  days == 0:
            Montag = Dienstag=Mittwoch=Donnerstag=Freitag=Samstag=Sonntag=np.nan
        else:
            special_case = 0
            for j, day in enumerate(days):
                
                if j> 6 and special_case== 0:
                    break
                # print("Pedro prints", i)
                if len(day.find("td", {"itemprop": "dayOfWeek"}).text.split()) == 0:
                    break
                if day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Montag":
                    Montag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Dienstag":
                    Dienstag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Mittwoch":
                    Mittwoch = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Donnerstag":
                    Donnerstag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Freitag":
                    Freitag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Samstag":
                    Samstag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                elif day.find("td", {"itemprop": "dayOfWeek"}).text.split()[0] == "Sonntag":
                    try:
                        Sonntag = day.find("time", {"itemprop": "opens"}).text + " - " + day.find("time", {"itemprop": "closes"}).text
                    except AttributeError:
                        Sonntag = "Closed"

        store_dict = {
            'street_number': street_number,
            'post_municipality': post_municipality,
            'telephone': telephone,
            'type_store': type_store,
            'Angebot':Angebot,
            'Montag': Montag,
            'Dienstag': Dienstag,
            'Mittwoch': Mittwoch,
            'Donnerstag': Donnerstag,
            'Freitag': Freitag,
            'Samstag': Samstag,
            'Sonntag': Sonntag,
            'gmaps_link': gmaps_link,
        }

        df = pd.concat([df, pd.DataFrame(store_dict, index=[0])], ignore_index=True)


    return df





In [232]:
stores_df =get_filialen(stores)

In [241]:
# stores_df

In [250]:
stores_df.at[58,"Sonntag"] = "Closed"
stores_df.at[58,"Sonntag"]

'Closed'

In [254]:
# stores_df.to_csv("../data/denner_branches/denner_branches.csv")

In [255]:
stores_df

,street_number,post_municipality,telephone,type_store,Angebot,Montag,Dienstag,Mittwoch,Donnerstag,Freitag,Samstag,Sonntag,gmaps_link
0,Wengistrasse 7,8004 Zürich,058 999 06 27,Denner Express,"Humidor, Bargeldbezug mit Post- / M-Card",08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=Wengi...
1,Molkenstrasse 8,8004 Zürich,058 999 14 13,Denner Express,"Humidor, Bargeldbezug mit Post- / M-Card",07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=Molke...
2,Werdstrasse 36,8004 Zürich,058 999 03 02,Denner Filiale,"Humidor, Bargeldbezug mit Post- / M-Card",08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=Werds...
3,Sihlhallenstrasse 5-7,8004 Zürich,058 999 04 19,Denner Express,"Humidor, Bargeldbezug mit Post- / M-Card, Post...",08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=Sihlh...
4,Badenerstrasse 296,8004 Zürich,058 999 14 75,Denner Filiale,"Humidor, Bargeldbezug mit Post- / M-Card",07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,07:00 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=Baden...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Römerstrasse 243,8404 Winterthur,058 999 14 62,Denner Express,"Humidor, Bargeldbezug mit Post- / M-Card",07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,08:00 - 19:00,Closed,https://maps.google.com/maps?saddr&daddr=R%C3%...
156,Hintermühlenstrasse 4,8409 Winterthur,058 999 13 15,Denner Filiale,"Humidor, Bargeldbezug mit Post- / M-Card",08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 19:00,Closed,https://maps.google.com/maps?saddr&daddr=Hinte...
157,Wässeristrasse 20,8340 Hinwil,058 999 06 67,Denner Filiale,"Humidor, Bargeldbezug mit Post- / M-Card",07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,07:30 - 20:00,Closed,https://maps.google.com/maps?saddr&daddr=W%C3%...
158,Wambisterstrasse 2,5412 Gebenstorf,058 999 13 40,Denner Filiale,"Humidor, Bargeldbezug mit Post- / M-Card",08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 20:00,08:00 - 18:00,Closed,https://maps.google.com/maps?saddr&daddr=Wambi...
